# 한글-Claude-v2 Model: Invoke Bedrock model using LangChain and a zero-shot prompt

---

### 중요
- 이 노트북은 Anthropic 의 Claude-v2 모델 접근 가능한 분만 실행 가능합니다. 
- 접근이 안되시는 분은 노트북의 코드와 결과 만을 확인 하시면 좋겠습니다.
- 만일 실행시에는 **"과금"** 이 발생이 되는 부분 유념 해주시기 바랍니다.

이 노트북에서는 LLM을 사용하여 지원 엔지니어로부터 받은 고객 서비스 품질에 대해 부정적인 피드백을 제공한 고객에게 이메일 응답을 생성하는 방법을 보여줍니다.

이 예제에서는 Bedrock에서 제공하는 Anthropic의 Claude 모델을 사용합니다. [LangChain](https://python.langchain.com/docs/get_started/introduction.html)과 통합된 Bedrock 버전을 사용합니다. LangChain은 언어 모델로 구동되는 애플리케이션을 개발하기 위한 프레임워크입니다. 이 프레임워크의 주요 측면을 통해 다양한 구성 요소를 함께 연결하여 고급 사용 사례를 생성함으로써 대규모 언어 모델을 보강할 수 있습니다.

이 노트북에서는 LangChain에서 제공하는 Bedrock API를 사용합니다. 이 예제에서 사용된 프롬프트는 프롬프트 외에 추가 컨텍스트를 제공하지 않기 때문에 제로샷 프롬프트라고 합니다.

**참고:** *이 노트북은 AWS 환경 내부 또는 외부에서 실행할 수 있습니다*.

#### 문맥
이전 예제 `00_generate_w_bedrock.ipynb`에서는 Boto3 클라이언트를 사용하여 Amazon Bedrock API와 통신하는 방법을 살펴보았습니다. 이 노트북에서는 유사한 사용 사례에 대해 LangChain 프레임워크를 활용하기 위해 조금 더 복잡하게 추가하려고 합니다. LangChain 프레임워크 내에서 Amazon Bedrock 통합의 사용과 'PromptTemplate'의 도움으로 텍스트를 생성하는 데 어떻게 사용할 수 있는지 살펴보겠습니다.

#### 패턴
추가 예제를 제공하지 않고 출력을 생성하기 위해 내부 모델에 대한 작업, 지침 및 입력으로 구성된 입력을 Amazon Bedrock API의 LangChain 구현에 제공하기만 하면 됩니다. 여기의 목적은 강력한 LLM이 당면한 작업을 쉽게 이해하고 매력적인 출력을 생성하는 방법을 보여주는 것입니다.

![](./images/bedrock_langchain.jpg)

#### 유스케이스
Amazon Bedrock에서 모델의 생성 기능을 시연하기 위해 이메일 생성 사용 사례를 살펴보겠습니다.

#### 페르소나
귀하는 AnyCompany의 고객 서비스 관리자인 권율이며 일부 고객은 고객 서비스에 만족하지 않고 고객 지원 엔지니어가 제공하는 서비스에 대해 부정적인 피드백을 제공하고 있습니다. 이제는 열악한 서비스에 대해 겸허히 사과하는 고객에게 응답하고 신뢰를 회복하고 싶습니다. 인간 친화적이고 이전 이메일 서신에서 고객의 정서에 맞게 개인화된 대량의 이메일을 생성하려면 LLM의 도움이 필요합니다.

#### 구현
이 사용 사례를 이행하기 위해 이 노트북에서는 고객의 이전 이메일을 기반으로 감사 메모가 포함된 이메일을 생성하는 방법을 보여줍니다. Amazon Bedrock LangChain 통합을 사용하는 Amazon Titan Text Large 모델을 사용합니다.

## 1. Bedrock Client 생성

In [15]:
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww


# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
# os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

print(colored("\n== FM lists ==", "green"))
pprint(bedrock_info.get_list_fm_models())

Create new client
  Using region: us-east-1
  Using profile: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)


NameError: name 'colored' is not defined

## 2. LangChain 통합을 사용하여 Bedrock 클라이언트 호출

langchain.llms 로 부터 Bedrock 클래스의 인스턴스를 만드는 것으로 시작하겠습니다. 이것은 Amazon Bedrock에서 사용할 수 있는 모델의 'model_id'를 예상합니다.

선택적으로 이전에 생성된 boto3 `client`와 `temperature`, `topP`, `maxTokenCount` 또는 `stopSequences`와 같은 매개변수를 보유할 수 있는 일부 `model_kwargs`를 전달할 수 있습니다(매개변수에 대한 자세한 내용은 Amazon Bedrock에서 탐색할 수 있음)


다른 모델은 다른 `model_kwargs`를 지원합니다.

In [3]:
from langchain.llms.bedrock import Bedrock

inference_modifier = {
    "max_tokens_to_sample": 4096,
    "temperature": 0.5,
    "top_k": 250,
    "top_p": 1,
    "stop_sequences": ["\n\nHuman"],
}

textgen_llm = Bedrock(
    model_id="anthropic.claude-v2",
    client=boto3_bedrock,
    model_kwargs=inference_modifier,
)

`클라이언트`를 LangChain에 전달함으로써 라이브러리가 이전에 구성을 확인한 것과 동일한 boto3 클라이언트를 사용하는지 확인할 수 있어야 합니다.

In [4]:
print(boto3_bedrock)
print(textgen_llm.client)

LangChain은 Amazon Bedrock API를 추상화하고 사용 사례를 쉽게 구축할 수 있도록 했습니다. 프롬프트를 전달할 수 있으며 자동으로 적절한 API로 라우팅되어 응답을 생성합니다. 텍스트 출력을 있는 그대로 가져오고 응답 본문에서 결과를 추출할 필요가 없습니다.



## 3.프롬프트 생성


고객 서비스 관리자가 고객에게 보낼 이메일을 생성하기 위한 프롬프트를 준비하겠습니다.

In [7]:
prompt = ("""Human:고객 서비스 관리자인 권율이 고객 "이순신"에게 이메일을 작성합니다.
이순신님은 고객 지원 엔지니어로 부터 제공하는 서비스에 대해 부정적인 피드백을 받았습니다.\n\nAssistant:""")

In [14]:
response = textgen_llm(prompt)
print_ww(response)

 안녕하세요 이순신님,

저는 권율입니다. 이순신님께서 최근 우리 회사의 고객 지원 서비스에 대해 불만족하셨다는 소식을 들었습니다.

고객님의 소중한 의견에 귀 기울이겠습니다. 우리 회사는 고객님의 만족을 최우선으로 생각하며, 여러분의 소리에 귀를 기울이고 개선점을 찾기 위해 노력하고 있습니다.

고객님께 불편을 드린 점 진심으로 사과드립니다. 이순신님의 경험을 통해 우리 서비스가 개선될 수 있도록 하겠습니다.

앞으로 고객님의 요구에 더 잘 부응할 수 있는 서비스를 제공하기 위해 최선을 다하겠습니다.

다시 한 번 지적해 주신 부분에 대해 감사드리며, 추후에도 소중한 의견을 주실 경우 언제든지 연락주세요.

감사합니다.

권율 드림


## 4. 결론

이제 Amazon Bedrock API에서 추상화 계층을 제공하는 LangChain 프레임워크를 사용하여 실험했습니다. 이 프레임워크를 사용하여 부정적인 피드백으로 인해 고객에게 응답하는 이메일을 생성하는 사용 사례를 확인했습니다.

### Next Action
- 프롬프트를 특정 사용 사례로 변경하고 다른 모델의 출력을 평가합니다.
- 다양한 매개변수를 사용하여 서비스의 대기 시간과 응답성을 이해하십시오.
- 더 나은 결과를 얻기 위해 다양한 프롬프트 엔지니어링 원칙을 적용합니다.